In [59]:
import numpy as np
import cv2 as cv
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

y_train = np.load("orientations_train.npy")
y_test = np.load("orientations_test.npy")

test_size = 1000
train_size = 10000
image_length = 4096
image_dim = 64

def vectorize_images(filename, data_size):
    X = np.empty(shape=(data_size, image_length))
   
    for i in range(data_size):     
        img = cv.imread("{filename}/{i}.jpg".format(filename = filename, i = i), cv.IMREAD_GRAYSCALE)
        X[i] = img.flatten()
        
    return X

X_train = vectorize_images("3dshapes_train" ,train_size)
X_test = vectorize_images("3dshapes_test" ,test_size)

__a)__ i)

In [60]:
# Select two orientations for binary classification
train_orients_set = list(set(y_train))
orient1 = train_orients_set[0]   # 0
orient2 = train_orients_set[1]   # 1

# Create binary label vector
y_train_bin = np.empty(train_size)
y_test_bin = np.empty(test_size)

y_train_bin = np.delete(y_train, np.argwhere( (y_train != orient1) & (y_train != orient2) ))
y_train_bin[y_train_bin == orient1] = 0
y_train_bin[y_train_bin == orient2] = 1
X_train_bin = X_train[(y_train == orient1) | (y_train == orient2)]

y_test_bin = np.delete(y_test, np.argwhere( (y_test != orient1) & (y_test != orient2) ))
y_test_bin[y_test_bin == orient1] = 0
y_test_bin[y_test_bin == orient2] = 1
X_test_bin = X_test[(y_test == orient1) | (y_test == orient2)]

"""print(X_train_bin)
print(len(X_train_bin))
print(y_train_bin)
print(len(y_train_bin))
print(X_test)
print(len(X_test))"""

# Standardize the data
scaler = StandardScaler()
X_scaled_bin_train = scaler.fit_transform(X_train_bin)
X_scaled_bin_test = scaler.transform(X_test_bin)

# Apply PCA to extract principal components
pca = PCA(n_components=10)  # n_components can be changed to extract more or fewer principal components as needed.
X_pca_bin_train = pca.fit_transform(X_scaled_bin_train)
X_pca_bin_test = pca.fit_transform(X_scaled_bin_test)

# Train SVM classifier
svm_bin = SVC(kernel='linear', C=0.1, random_state=42)
svm_bin.fit(X_pca_bin_train, y_train_bin)

# Predict labels for validation set
y_pred_bin_test = svm_bin.predict(X_pca_bin_test)

# Compute validation accuracy
acc_bin_test = accuracy_score(y_test_bin, y_pred_bin_test)
print("Binary classification accuracy:", acc_bin_test)


Binary classification accuracy: 0.6062992125984252
